Imports:

requests for our scraper.<br>
beautiful soup for our scraper.<br>
re in case we need regex<br>
pandas to build a data frame for our q and a<br>
json to save our q and a files because it is more flexible than just a text file

In [0]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json

Load up our JSONs

In [0]:
# with open('/content/drive/My Drive/Colab Datasets/COVID Chat Bot/answers.json', 'r') as f:
#   answers = json.load(f)

In [0]:
# with open('/content/drive/My Drive/Colab Datasets/COVID Chat Bot/questions.json', 'r') as f:
#   questions = json.load(f)

A zip to put our questions and answers together in a list as a tuple

In [0]:
# q_and_a = list(zip(questions, answers))

And put it into a dataframe

In [0]:
# df = pd.DataFrame(q_and_a, columns =['Questions', 'Answers'])
# df.head(3)

,Questions,Answers
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza..."
2,Why might someone blame or avoid individuals a...,People in the U.S. may be worried or anxious a...


and that is our finished work for this part of the process!

everything below the line represents all the other things including the scraper, trial and error, and asking for help on things i had not yet done before.

#-----------------------------------------------------------------------

Our url for this notebook is the CDC faq on Coronavirus<br>
requests is how we grab the url, with some little tests we commented out after ffinding that we got all we needed.<br>
then we assigned the response to the variable 'data'.<br>
<br>
response 200 means our request was successful. yay!

In [0]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/faq.html'
response = requests.get(url)
print(response)
# make sure we got a valid response
if(response.ok):
  # get the full data from the response
  data = response.text
  print(data)

We set up our soup (called instantiation) with the data and either html or lxml. try them both and see which works better, so says Dennis.<br>
Sometimes we will read in the tags we want if everything is sufficiently spaced. In our case it is not so we are going to build our tags as we go.

In [0]:
soup = BeautifulSoup(data, 'lxml')
TAGS = ['span']
# including the div tag gave a lot more fluff stuff
# i need to filter further so it only grabs headings from div id=accordion-number

These helped us figure out how to grab what we wanted. Once we got it, commented them out and set up our loops below.

In [0]:
text_q = [element.get_text() for element in soup.find_all('span') if soup.find(id='accordion-15') and soup.find(role='heading')]
questions = ' '.join(text_q)

text_a = [element.get_text() for element in soup.find_all('p') if soup.find(id='accordion-15') and soup.find(id='accordion-15-card-1')]
answers = ' '.join(text_a)

This totally did not work how I wanted it to. Saved for the same reason we don't remove amendments from the Constitution: remember.

In [0]:
# all_things = soup.find_all("div", id=re.compile("^accordion"))
# everything = questions = ' '.join(all_things)

TypeError: ignored

This gave me my elements, so getting closer on building the loop!

In [0]:
for element in soup.find_all('div', class_='card-body'):
  print(element)

And now we have successfully isolated the questions!

In [0]:
soup.find('div', id='accordion-15').find_all('span', role='heading')

[<span aria-level="1" role="heading">What is the source of the virus?</span>,
 <span aria-level="1" role="heading">How does the virus spread?</span>,
 <span aria-level="1" role="heading">Why are we seeing a rise in cases?</span>,
 <span aria-level="1" role="heading">Can someone who has had COVID-19 spread the illness to others?</span>,
 <span aria-level="1" role="heading">Can someone who has been quarantined for COVID-19 spread the illness to others?</span>,
 <span aria-level="1" role="heading">Can the virus that causes COVID-19 be spread through food, including restaurant take out, refrigerated or frozen packaged food?</span>,
 <span aria-level="1" role="heading">Can I get sick with COVID-19 if it is on food?</span>,
 <span aria-level="1" role="heading">Will warm weather stop the outbreak of COVID-19?</span>,
 <span aria-level="1" role="heading">What is community spread?</span>,
 <span aria-level="1" role="heading">What temperature kills the virus that causes COVID-19?</span>,
 <span 

So here we build our for loop to scrape the site:

first we create an empty list that will hold our text.<br>
then we set a range that matches the accordion names within the html. we will start at 15, like they do, and end at 28, which is why we use 29, because it is not inclusive (the range does not count the last number, so always go +1)<br>
now we set up our scraper. we want soup to find tags 'div' with the id 'accordion-number'. The i represents the range established above. So our first id will be accordion-15, and the last, because not inclusive, will be accordion-28. we use the f string to help us filter so we don't grab every accordion.<br>
then we want to scrape all (find_all) the 'span' tags with the role 'heading' because that is where the text is stored.<br>
our next for loop, now that we have isolated our text, is go through the elements in each 'accordion', 15-28, and scrape the text from the span tag containing the role 'heading' and assign it to the variable text_q, for text questions.<br>
lastly, we want to assign that text back to our empty list, questions, outside the for loops.<br>
because this is iterative (it loops until it has interated over all of the chosen elements), it will keep overwriting text_q with the next question. that is why the last line assigns the text back to questions using the append function.

In [0]:
questions = []
for i in range(14,27):
  q_s = soup.find('div', id=f'accordion-{i}').find_all('span', role='heading')
  for element in q_s:
    text_q = element.get_text()
    questions.append(text_q)

Ledz take a looksie at what we gotchea

In [0]:
# questions

awesome. we do it again, but this time for answers.<br>
the main difference is we are dealing with a class now instead of id and role, so we have to select instead of find.

In [0]:
answers = []
for i in range(14,27):
  a_s = soup.find('div', id=f'accordion-{i}').select('div[class*=card-body]')
  for element in a_s:
    text_a = element.get_text()
    answers.append(text_a)

In [0]:
# answers

Now we check the len of our variables to make sure they match up

In [0]:
print(len(questions))
print(len(answers))

74
75


In [0]:
answers.pop(39)

'If you develop emergency warning signs for COVID-19 get medical attention immediately. Emergency warning signs include*:\n\nTrouble breathing\nPersistent pain or pressure in the chest\nNew confusion or inability to arouse\nBluish lips or face\n\n*This list is not all inclusive. Please consult your medical provider for any other symptoms that are severe or concerning.\n'

In [0]:
with open('answers.json', 'w') as f:
  json.dump(answers, f)

In [0]:
with open('questions.json', 'w') as f:
  json.dump(questions, f)

So...uh oh on that. But before we get to that code, here is another place we tested grabbing just the text using a for loop.

In [0]:
# for element in answers2:
#   print(element.get_text())
#   print('--------------')

testing the select function

In [0]:
# soup.select('div[class*=card-body]')

testing final_all with regex's .compile<br>
iirc, the carot '^' means including

In [0]:
# soup.find_all("div", id=re.compile("^accordion"))

checking if questions grabbed what we wanted. and it did. but also stuff we didn't wanted.

In [0]:
# questions

' Self-Checker Coronavirus Self-Checker × Centers for Disease Control and Prevention. CDC twenty four seven. Saving Lives, Protecting People Centers for Disease Control and Prevention. CDC twenty four seven. Saving Lives, Protecting People Search Search Icon  Close Icon Ã\x97 Submit For a full list of topics:  Section Navigation    CDC Home Facebook  Twitter  LinkedIn  Email  Syndicate  external icon  external icon   What is a novel coronavirus? Why is the disease being called coronavirus disease 2019, COVID-19? external icon  Why might someone blame or avoid individuals and groups (create stigma) because of COVID-19? How can people help stop stigma related to COVID-19?  What is the source of the virus? How does the virus spread? Can someone who has had COVID-19 spread the illness to others? Can someone who has been quarantined for COVID-19 spread the illness to others? Can the virus that causes COVID-19 be spread through food, including refrigerated or frozen food? Will warm weather s

samesies

In [0]:
# answers

'Follow Official Sources for Accurate Information!\nHelp control the spread of rumors. Visit FEMA’s rumor control pageexternal icon.\nBeware of fraud schemes related to the novel coronavirus (COVID-19). Visit Office of Inspector General’s COVID-19 fraud alert pageexternal icon. A novel coronavirus is a new coronavirus that has not been previously identified. The virus causing coronavirus disease 2019 (COVID-19), is not the same as the coronaviruses that commonly circulate among humans\xa0and cause mild illness, like the common cold. A diagnosis with coronavirus 229E, NL63, OC43, or HKU1 is not the same as a COVID-19 diagnosis. Patients with COVID-19 will be evaluated and cared for differently than patients with common coronavirus diagnosis. On February 11, 2020 the World Health Organization announced an official name for the disease that is causing the 2019 novel coronavirus outbreak, first identified in Wuhan China. The new name of this disease is coronavirus disease 2019, abbreviated

a list of things i need to do. as usual, i forgot sleep.

In [0]:
# ingest, preprocess
# create a dictionary with questions(keys) and answers(values)
# vectorize
# 

Read in our newly saved files so as not to pester the CDC

In [0]:
answers = pd.read_json('/content/drive/My Drive/Colab Datasets/COVID Chat Bot/answers.json')
questions = pd.read_json('/content/drive/My Drive/Colab Datasets/COVID Chat Bot/questions.json')
print(len(questions))
print(len(answers))

# ultimately this will work differently since i read them in with pandas
# since they are being read in as dataframes, we'll need a join

69
69


how to save a list to a json

In [0]:
with open("test.json", 'w') as f:
  json.dump(q_and_a, f)

how to load a list from a json

In [0]:
with open("test.json", 'r') as f:
  new_data = json.load(f)

doing the same to save questions and answers so we don't repeatedly scrape the CDC (oops!)

In [0]:
with open('answers.json', 'w') as f:
  json.dump(answers, f)

In [0]:
with open('questions.json', 'w') as f:
  json.dump(questions, f)

Remember that our lens were off by 1? After researching I found that an answer in 27 was duplicated in 28, so we pop 28 to ditch it.

In [0]:
answers.pop(28)

'If you develop emergency warning signs for COVID-19 get medical attention immediately. Emergency warning signs include*:\n\nTrouble breathing\nPersistent pain or pressure in the chest\nNew confusion or inability to arouse\nBluish lips or face\n\n*This list is not all inclusive. Please consult your medical provider for any other symptoms that are severe or concerning.\n'

check the len of answers, and it is correctamundo

In [0]:
len(answers)

69

The following is a lot of Dennis saying "make a new cell" while i was trying to isolate the problem.

In [0]:
new_data

In [0]:
find_problem = q_and_a[27]

In [0]:
answers[27]

In [0]:
answers.pop(28)

at one point i thought the problem was that the answer for 27 was broken in two, and the second half assigned to 28. so here were added 28 to 27. which is when we found it was actually a duplicate for some reason.

In [0]:
answers[27] = answers[27]+answers[28]

In [0]:
print(answers[27])

And here we make a dataframe that holds our q and a. it is isn't finished yet. we need to do some text preprocessing. but we have our data, saved it, and can move on to another notebook to handle that so we keep things short and manageable.

this did not work because i read in my JSONs as dataframes, meaning i would need to join them rather than assign a list of tuples to a frame.

In [0]:
q_and_a

[(0, 0)]

In [0]:
df = pd.DataFrame(q_and_a)

In [0]:
df

,0,1
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza..."
2,Why might someone blame or avoid individuals a...,People in the U.S. may be worried or anxious a...
3,How can people help stop stigma related to COV...,"People can fight stigma and help, not hurt, ot..."
4,What is the source of the virus?,Coronaviruses are a large family of viruses. S...
...,...,...
64,"While school’s out, can my child hang out with...",\nThe key to slowing the spread of COVID-19 is...
65,"While school’s out, how can I help my child co...",\nStay in touch with your child’s school.\n\nM...
66,"While school’s out, will kids have access to m...",\nCheck with your school on plans to continue ...
67,"While school’s out, how can I keep my family h...",\nWatch your child for any signs of illness. \...


save the dataframe

In [0]:
df.to_csv('cdc_qa.csv', index=False)